In [5]:
import cloudpickle as pkl
import pandas as pd
import os


with open('/home/antoine/Documents/Projets_DEV_IA/Allocine_scraping/FastAPI/Projet_API/Movie_Predict/app/data/model_bundle_final.pkl', 'rb') as f:
    bundle = pkl.load(f)
    model = bundle['model']
    columns = bundle['features']
print(50*'*')
print(columns)
path = "/home/antoine/Documents/Projets_DEV_IA/Allocine_scraping/FastAPI/Projet_API/Movie_Predict/app/data"
# test = os.listdir(path)
weekly_scrapy = os.listdir(path)[0]
# print(50*'*')
# print(path)
# print(weekly_scrapy)
movies_pred = pd.read_csv(path+'/'+weekly_scrapy)
movies_pred_list = movies_pred.copy()
movies_pred_list['predictions'] = 0

movies_pred['critics_rating'] = movies_pred['critics_rating'].str.replace(',', '.')
movies_pred['critics_rating'] = pd.to_numeric(movies_pred['critics_rating'], errors='coerce')
movies_pred['critics_rating'][movies_pred['critics_rating'].isna()] = 3
movies_pred['duration'][movies_pred['duration'].isnull()] = 102

for col in columns:
    if col not in movies_pred.columns:
        movies_pred[col] = 0

i = 0
for cat in movies_pred['categories'].iloc[:]:
    test = cat.split(',')
    for c in test:
        if c in columns:
            movies_pred[c].loc[i] = 1
    i += 1


i = 0
for date in movies_pred['release_date'].iloc[:]:
    if type(date) == str:
        d = date.split(' ')
        if d[2] in columns:
            movies_pred[d[2]].loc[i] = 1
    i += 1

i = 0
for date in movies_pred['release_date'].iloc[:]:
    if type(date) == str:
        d = date.split(' ')
        if d[1] in columns:
            movies_pred[d[1]].loc[i] = 1
    i += 1

i = 0
for count in movies_pred['country'].iloc[:]:
    if type(count) == str:
        test = count.split(',')
        for c in test:
            if c in columns:
                movies_pred[c].loc[i] = 1
    i += 1

i = 0
for dir in movies_pred['casting'].iloc[:]:
    if type(dir) != float:
        test = dir.split(',')
        for t in test:
            if t in columns:
                movies_pred[t].loc[i] = 1
    i+=1

movies_pred['shortfilm']= [x if x < 80 else 0 for x in movies_pred['duration']]
movies_pred['medfilm']= [x if x >= 80 and x <=130 else 0 for x in movies_pred['duration']]
movies_pred['longfilm']= [x if x > 130 and x<=160 else 0 for x in movies_pred['duration']]
movies_pred['verylongfilm']= [x if x > 160 else 0 for x in movies_pred['duration']]
movies_pred_df = movies_pred[columns]

y_pred = model.predict(movies_pred_df)
movies_pred_list['predictions'] = y_pred
weekly_scrapy = weekly_scrapy.replace('.csv','')
pred_df = movies_pred_list[(movies_pred_list['release_date']=='2 avril 2025') & (movies_pred_list['production_year']>2023)].sort_values('predictions',ascending=False).head(10)
# print(pred_df)


**************************************************
['Tom Cruise' 'Jean Dujardin' 'Leonardo DiCaprio' 'Robert Downey Jr.'
 'Johnny Depp' 'Franck Dubosc' 'Vin Diesel' 'Will Smith'
 'Christian Clavier' 'Brad Pitt' 'Gérard Lanvin' 'Philippe Lacheau'
 'Dwayne Johnson' 'Jean Reno' 'Matt Damon' 'Daniel Radcliffe'
 'Guillaume Canet' 'José Garcia' 'George Clooney' 'Nicolas Cage'
 'Tom Hanks' 'Kev Adams' 'Alain Chabat' 'Gad Elmaleh' 'Romain Duris'
 'Angelina Jolie' 'Omar Sy' 'Chris Pratt' 'Mark Wahlberg' 'Christian Bale'
 'Bruce Willis' 'Hugh Jackman' 'Dany Boon' 'Ben Stiller' 'François Cluzet'
 'Ryan Reynolds' 'James McAvoy' 'Jennifer Lawrence' 'Ben Affleck'
 'Audrey Tautou' 'Keanu Reeves' 'Jim Carrey' 'Daniel Craig'
 'Kristen Stewart' 'Daniel Auteuil' 'Vincent Cassel' 'Lambert Wilson'
 'Guillaume Gallienne' 'Sophie Marceau' 'Gérard Jugnot' 'Kad Merad'
 'Liam Neeson' 'Chris Hemsworth' 'Jean-Paul Rouve' 'Tom Hardy'
 'Karin Viard' 'Tom Holland' 'Sylvester Stallone' 'Cécile de France'
 'Joaquin Ph

/tmp/ipykernel_56843/513050282.py:24: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  movies_pred['critics_rating'][movies_pred['critics_rating'].isna()] = 3
/tmp/ipykernel_56843/513050282.py:24: SettingWithCopyWarning: 
A value is trying to b

In [6]:
print(pred_df)

                    awards budget  \
19                     NaN      -   
5                      NaN      -   
27  1 prix et 1 nomination      -   
16                     NaN      -   
0                      NaN      -   
11                     NaN      -   
24                     NaN      -   
6                      NaN      -   
1                      NaN      -   
25                     NaN      -   

                                              casting  \
19  Philippe Mechelen,Hakim Jemili,Christian Clavi...   
5   Jared Hess,Jack Black,Jason Momoa,Danielle Bro...   
27                                                NaN   
16                                                NaN   
0                                                 NaN   
11  Clément Schneider,Francis Leplay,Pierre-Franço...   
24                                                NaN   
6   Jean-Claude Barny,Déborah François,Stanislas M...   
1                                                 NaN   
25                    